<a href="https://colab.research.google.com/github/LavanyaPobbathi/Lavanya_INFO5731_Fall2023/blob/main/5731_Final_Project/Data%20Cleaning/patent_webscraping_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display, HTML

# List of patent numbers
patent_numbers = ["US9667969"]

patent_data = {}

for patent_number in patent_numbers:
    url = f"https://patents.google.com/patent/{patent_number}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    # Get abstract text
    abstract_texts = soup.find_all('div', class_='abstract')
    TEXT_abstract = []

    for abstract_text in abstract_texts:
        TEXT_abstract.append(abstract_text.get_text())
        patent_data[patent_number] = {
            'Abstract': ' '.join(TEXT_abstract).strip(),
            'Claims': None,  # Placeholder for claims text
            'Description': None,  # Placeholder for description text
            'Images': []  # Placeholder for image URLs
        }

    # Get claim text
    claim_texts = soup.find_all('div', class_='claim-text')
    TEXT_claim = []

    for claim_text in claim_texts:
        TEXT_claim.append(claim_text.get_text())
        patent_data[patent_number]['Claims'] = ' '.join(TEXT_claim).strip()

    # Get description text
    description_texts = soup.find_all('div', class_='description')
    TEXT_description = []

    for description_text in description_texts:
        TEXT_description.append(description_text.get_text())
        patent_data[patent_number]['Description'] = ' '.join(TEXT_description).strip()

    # Get patent images
    image_urls = []
    image_tags = soup.find_all('img')

    for image_tag in image_tags:
        image_url = image_tag.get('src')
        if image_url:
            image_urls.append(image_url)

    # Add image URLs to the patent's dictionary
    patent_data[patent_number]['Images'] = image_urls

    # Print patent data
    print(f"Patent Number: {patent_number}")
    print("Abstract Text:")
    print(patent_data[patent_number]['Abstract'])
    print("")
    print("Claims Text:")
    print(patent_data[patent_number]['Claims'])
    print("")
    print("Description Text:")
    print(patent_data[patent_number]['Description'])
    print("")

    # Display images
    for image_url in image_urls:
      print("Image URLs and figures:")
      print(image_url)
      print("")
      display(HTML(f'<img src="{image_url}" alt="Image">'))
      print("")


Patent Number: US9667969
Abstract Text:
Method for encoding a video stream having a transparency information channel in view of a predetermined target bit rate for said transparency information channel, comprising: tentatively applying a first encoding (130) to said transparency information channel, said first encoding (130) comprising lossless vector graphics encoding; if said first encoding results in an encoded bitrate within said predetermined target bitrate (140), selecting said first encoding (150) to produce an encoded version of said transparency information channel; otherwise, tentatively applying a second encoding (160) to said transparency information channel, said second encoding (160) comprising a mathematical representation encoding scheme; if said second encoding results in an encoded bitrate within said predetermined target bitrate (170), selecting said second encoding (180) to produce an encoded version of said transparency information channel; otherwise, selecting a t


Image URLs and figures:
https://patentimages.storage.googleapis.com/b9/51/fb/1a25a48b6d0725/US09667969-20170530-D00001.png




Image URLs and figures:
https://patentimages.storage.googleapis.com/ee/80/c9/e80c0e57a69b78/US09667969-20170530-D00002.png



Code for Extracting the patent dataset and storing them in a csv file

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Read the Excel file containing patent numbers
input_excel = 'PromptData.xlsx'  # Replace with your Excel file path
patent_numbers_df = pd.read_excel(input_excel)
patent_numbers = patent_numbers_df['Patent. Number'].tolist()  # Replace 'PatentNumberColumn' with the actual column name

patent_data = []

for patent_number in patent_numbers:
    try:
        url = f"https://patents.google.com/patent/{patent_number}"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")

        # Initialize dictionary for each patent
        patent_info = {'Patent Number': patent_number, 'Abstract': None, 'Claims': None, 'Description': None}

        # Get abstract text
        abstract = soup.find('div', class_='abstract')
        if abstract:
            patent_info['Abstract'] = abstract.get_text().strip()

        # Get claim text
        claims = soup.find_all('div', class_='claim-text')
        if claims:
            patent_info['Claims'] = ' '.join([claim.get_text().strip() for claim in claims])

        # Get description text
        description = soup.find('div', class_='description')
        if description:
            patent_info['Description'] = description.get_text().strip()

        patent_data.append(patent_info)
    except Exception as e:
        print(f"Error processing patent number {patent_number}: {e}")

# Convert the list of dictionaries to a DataFrame
patent_data_df = pd.DataFrame(patent_data)

# Save the DataFrame to a new Excel file
output_excel = 'patent_data.xlsx'  # Replace with your desired output file path
patent_data_df.to_excel(output_excel, index=False)

print("Patent data extraction completed.")
